In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
sn.set()
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [35]:
seg_df=pd.read_csv("segmentation data.csv")
segment_df=seg_df.drop("ID",axis=1)
segment_df.head()

,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
0,0,0,67,2,124670,1,2
1,1,1,22,1,150773,1,2
2,0,0,49,1,89210,0,0
3,0,0,45,1,171565,1,1
4,0,0,53,1,149031,1,1


In [36]:
seg_df.describe()

,ID,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
count,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000
mean,1.000010e+08,0.457000,0.496500,35.909000,1.03800,120954.419000,0.810500,0.739000
std,5.774946e+02,0.498272,0.500113,11.719402,0.59978,38108.824679,0.638587,0.812533
min,1.000000e+08,0.000000,0.000000,18.000000,0.00000,35832.000000,0.000000,0.000000
25%,1.000005e+08,0.000000,0.000000,27.000000,1.00000,97663.250000,0.000000,0.000000
50%,1.000010e+08,0.000000,0.000000,33.000000,1.00000,115548.500000,1.000000,1.000000
75%,1.000015e+08,1.000000,1.000000,42.000000,1.00000,138072.250000,1.000000,1.000000
max,1.000020e+08,1.000000,1.000000,76.000000,3.00000,309364.000000,2.000000,2.000000


In [11]:
cols=[i for i in seg_df.columns if seg_df[i].nunique() <5]
dic=dict()
for i in cols :
    print(i)
    dic[i]=seg_df[i].unique()

Sex
Marital status
Education
Occupation
Settlement size


In [12]:
dic

{'Sex': array([0, 1], dtype=int64),
 'Marital status': array([0, 1], dtype=int64),
 'Education': array([2, 1, 0, 3], dtype=int64),
 'Occupation': array([1, 0, 2], dtype=int64),
 'Settlement size': array([2, 0, 1], dtype=int64)}

In [20]:
maps={
    "Settlements size":{0:"Small",1:"Medium",2:"Large"},
    "Occupation":{0:"Skilled",1:"Manager",1:"Owner"},
    "Education":{0:"12th",1:"Bachelors",2:"Masters",3:"Phd"},
    "Marital status":{0:"Single",1:"Married"},
    "Sex":{0:"Female",1:"Male"}
}

In [21]:
reverse_maps={i:{v:k for k,v in mapping.items()} for i,mapping in maps.items()}

In [22]:
reverse_maps

{'Settlements size': {'Small': 0, 'Medium': 1, 'Large': 2},
 'Occupation': {'Skilled': 0, 'Owner': 1},
 'Education': {'12th': 0, 'Bachelors': 1, 'Masters': 2, 'Phd': 3},
 'Marital status': {'Single': 0, 'Married': 1},
 'Sex': {'Female': 0, 'Male': 1}}

In [4]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [5]:
x_scaled=scaler.fit_transform(segment_df)
x_scaled.shape

(2000, 7)

In [26]:
from sklearn.decomposition import PCA
pca=PCA(n_components=0.80,random_state=64)

In [7]:
pca_df=pd.DataFrame(pca.fit_transform(x_scaled))
pca_df

,0,1,2
0,2.514746,0.834122,2.174806
1,0.344935,0.598146,-2.211603
2,-0.651063,-0.680093,2.280419
3,1.714316,-0.579927,0.730731
4,1.626745,-0.440496,1.244909
...,...,...,...
1995,-0.866034,0.298330,1.438958
1996,-1.114957,0.794727,-1.079871
1997,-1.452298,-2.235937,0.896571
1998,-2.241453,0.627108,-0.530456


In [13]:
from sklearn.cluster import KMeans
model_kmeans=KMeans(n_clusters=4,random_state=42,max_iter=1000, tol=0.0001)
k_means_cluster=model_kmeans.fit_predict(pca_df)

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler

In [28]:
column_transformer=Pipeline(
        steps=[
            ('imputer',SimpleImputer(strategy="median")),
            ('scaler',StandardScaler()),
            ('pca',PCA(n_components=0.80,random_state=64))
        ])


In [29]:
X=segment_df

In [30]:
# Unsupervised clustering pipeline
clustering_pipeline = Pipeline([
    ('kmeans', KMeans(n_clusters=4,random_state=42,max_iter=1000, tol=0.0001))
])

NameError: name 'KMeans' is not defined

In [31]:
# Fit and transform the data
ext=pd.DataFrame(column_transformer.fit_transform(X))

In [34]:
column_transformer.feature_names_in_

array(['Sex', 'Marital status', 'Age', 'Education', 'Income',
       'Occupation', 'Settlement size'], dtype=object)

In [34]:
new_clutsers=clustering_pipeline.fit_predict(ext)

In [35]:
sum(k_means_cluster==new_clutsers)

2000

In [37]:
maps={0:"Standard",1:"Platinum",2:"Gold",3:"Bronze"}

In [38]:
model_kmeans= KMeans(n_clusters=4,random_state=42,max_iter=1000, tol=0.0001)

In [39]:
model_kmeans.fit(ext)

KMeans(max_iter=1000, n_clusters=4, random_state=42)

In [40]:
import pickle
# Scaler 
pickle.dump(column_transformer, open('column_transformer.pickle', 'wb'))
# KMeans PCA 
pickle.dump(model_kmeans, open('model_kmeans.pickle', 'wb'))

In [41]:
model_features=['Sex', 'Marital status', 'Age', 'Education', 'Income', 'Occupation', 'Settlement size']